# SNEWS_PT Examples

In [1]:
# Global params 
from datetime import datetime

def nu_t():
    return datetime.utcnow().strftime("%H:%M:%S:%f")

my_detector = 'DS-20K'

## Publication

In [3]:
# %%timeit
from SNEWS_PT.snews_pub import CoincidenceTier, Publisher

with Publisher() as pub:
    message = CoincidenceTier(detector_name=my_detector, neutrino_time=nu_t(), p_value = 0.98,).message()
    pub.send(message)

---------------------------------------------------------
_id                 :18_CoincidenceTier_21/12/16_17:11:53:875996
detector_name       :DS-20K
sent_time           :21/12/16 17:11:53
machine_time        :None
neutrino_time       :17:11:53:847107
p_value             :0.98


In [4]:
%%time
from SNEWS_PT.snews_pub import SignificanceTier, Publisher

with Publisher() as pub:
    message = SignificanceTier(detector_name=my_detector, 
                               p_values=[0.78,0.67,0.75,0.56],
                               neutrino_time = nu_t()).message()
    pub.send(message)


---------------------------------------------------------
_id                 :18_SigTier_21/12/16_17:12:13:889697
detector_name       :DS-20K
sent_time           :21/12/16 17:12:13
machine_time        :None
neutrino_time       :17:12:13:864437
p_values            :[0.78, 0.67, 0.75, 0.56]
CPU times: user 52 ms, sys: 34.5 ms, total: 86.5 ms
Wall time: 3.28 s


In [5]:
from SNEWS_PT.snews_pub import TimingTier, Publisher

with Publisher() as pub:
    message = TimingTier(detector_name=my_detector, 
                         neutrino_time=nu_t(), 
                         timing_series = ['16:23:24"4546','16:23:24"4666','16:23:24"4756']).message()
    pub.send(message)

---------------------------------------------------------
_id                 :18_TimeTier_21/12/16_17:12:30:175435
detector_name       :DS-20K
sent_time           :21/12/16 17:12:30
machine_time        :None
neutrino_time       :17:12:30:143090
timing_series       :['16:23:24"4546', '16:23:24"4666', '16:23:24"4756']


In [6]:
from SNEWS_PT.snews_pub import Retraction, Publisher

with Publisher() as pub:
    message = Retraction(detector_name = my_detector, 
                         which_tier='CoincidenceTier', 
                         n_retract_latest=5, 
                         retraction_reason='someone spilled coffee on the DAQ').message()
    pub.send(message)

---------------------------------------------------------
IT'S OKAY, WE ALL MAKE MISTAKES
_id                 :18_FalseOBS_21/12/16_17:12:39:511025
detector_name       :DS-20K
sent_time           :21/12/16 17:12:39
machine_time        :None
false_id            :None
which_tier          :CoincidenceTier
N_retract_latest    :5
retraction_reason   :someone spilled coffee on the DAQ


## Subscription

In [2]:
from SNEWS_PT.snews_sub import Subscriber

Subscriber().subscribe()

You are subscribing to ALERT
Broker:kafka://kafka.scimma.org/snews.alert-test
Done
